## ETHZ-03-01-01 - Mapping landslides with Deep Learning algorithms applied to EO data

This application takes Sentinel-2 and ALO DEM to generate a landslide mask

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Landslide mapping with Deep Learning algorithms applied to EO data'),
                ('abstract', 'This application takes Sentinel-2 and ALO DEM to generate a landslide mask'),
                ('id', 'ewf-notebook-stagein-2')])

### Parameter Definition

In [ ]:
#model = dict([('id', 'model'),
#            ('value', 'Oregon'),
#            ('title', 'Chose the model trained in: Oregon or Bhutan'),
#            ('abstract', 'Set the value of WKT Polygon')])

In [ ]:
model = dict([('id', 'model'),
            ('value', 'Bhutan'),
            ('title', 'Chose the model trained in: Oregon or Bhutan'),
            ('abstract', 'Set the value of WKT Polygon')])

In [ ]:
#aoi = dict([('id', 'aoi'),
#            ('value', 'POLYGON ((456037.4350113738 4837015.1639622, 456037.4350113738 4855174.452151849, 425620.7027991246 4855174.452151849, 425620.7027991246 4837015.1639622, 456037.4350113738 4837015.1639622))'),
#            ('title', 'WKT Polygon for the Bounding Box in EPSG:32610 (Oregon model) or EPSG:32646 (Bhutan model) '),
#            ('abstract', 'Set the value of WKT Polygon according to the model chosen')])

In [ ]:
aoi = dict([('id', 'aoi'),
            ('value', 'POLYGON ((231100.37601192091824487 3032459.54463443346321583, 208550.73706856259377673 3032867.78160779643803835, 208550.73706856259377673 3097148.90503269853070378, 231100.37601192091824487 3096740.66805933555588126, 231100.37601192091824487 3032459.54463443346321583))'),
            ('title', 'WKT Polygon for the Bounding Box in EPSG:32610 (Oregon model) or EPSG:32646 (Bhutan model) '),
            ('abstract', 'Set the value of WKT Polygon according to the model chosen')])

### <a name="runtime">Runtime parameter definition

**Input references**

This is the Sentinel-1 stack catalogue references

In [ ]:
#input_references = ('https://catalog.terradue.com/sentinel2/search?format=atom&uid=S2B_MSIL1C_20200412T185909_N0209_R013_T10TDP_20200412T221853', 'https://catalog.terradue.com/alos-dem/search?format=atom&uid=N043W124')

In [ ]:
input_references = ('https://catalog.terradue.com/sentinel2/search?format=atom&uid=S2B_MSIL1C_20191116T044039_N0208_R033_T46RBR_20191116T072925', 'https://catalog.terradue.com/alos-dem/search?format=atom&uid=N027E090') 

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = "/workspace/data"

**Local path**

This path defines where the full path to each data product. 

In [ ]:
import os

In [ ]:
#local_data = ('/workspace/data/S2B_MSIL1C_20200412T185909_N0209_R013_T10TDP_20200412T221853', '/workspace/data/N043W124')

In [ ]:
local_data = ('/workspace/data/S2B_MSIL1C_20191116T044039_N0208_R033_T46RBR_20191116T072925', '/workspace/data/N027E090')

## <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
import os
import sys
import shutil
sys.path.append(os.getcwd())
sys.path.append('/application/notebook/libexec/')

import helpers as nx

# Bin folder for the SAGA-GIS installation
SAGA_PATH = "/usr/local/bin" 
SAGA_PATH = "/home/davidcordeiro/saga/bin" 
OG_PATH = os.environ['PATH']
if SAGA_PATH not in OG_PATH:
    os.environ['PATH'] = "{}:{}".format(SAGA_PATH, OG_PATH)

import cioppy
ciop = cioppy.Cioppy()

## <a name="stage-in">Stage-In

In [ ]:
for path, reference in zip(local_data, input_references):
    search = ciop.search(end_point = reference,
                         params = [],
                         output_fields='platform', 
                         model='EOP')
    platform = search[0]['platform']
    if platform == 'S2B' or platform == 'S2A':
        SOURCE_SENTINEL_FOLDER = path
    elif platform == 'ALOS':
        SOURCE_DEM_FOLDER = path

##### DEM input

In [ ]:
dsm_identifier = [f for f in os.listdir(SOURCE_DEM_FOLDER) if (os.path.isfile(os.path.join(SOURCE_DEM_FOLDER, f)) and f.endswith('DSM.tif'))]
SOURCE_DEM_PATH = os.path.join(SOURCE_DEM_FOLDER, dsm_identifier[0])

##### Sentinel input

In [ ]:
s2_identifier          = os.path.basename(SOURCE_SENTINEL_FOLDER)
SOURCE_S2_SAFE         = os.path.join(SOURCE_SENTINEL_FOLDER, s2_identifier + ".SAFE")
s2_granule             = os.path.join(SOURCE_S2_SAFE, 'GRANULE')
s2_interfolder         = [name for name in os.listdir(s2_granule)][0]
SOURCE_S2_IMAGE_FOLDER = os.path.join(s2_granule, s2_interfolder, 'IMG_DATA')

## Pre-Processing constants

In [ ]:
region = model['value']
if region == 'Oregon':
    PROJECT_EPSG      = 32610
    model_name        = 'trainedModelOregon.hdf5'
    S2_RESOLUTION = [7]
    WORK_RES = 27
elif region == 'Bhutan':
    PROJECT_EPSG      = 32646
    model_name        = 'trainedModelBhutan.hdf5'
    S2_RESOLUTION = [10]
    WORK_RES = 30

##### Folders paths

In [ ]:
FEATURES_PATH     = os.path.join(data_path, 'derivedFeatures', 'features')
OPTICAL_PATH      = os.path.join(data_path, 'derivedFeatures', 'fromOptical')
DEM_PATH          = os.path.join(data_path, 'derivedFeatures', 'multiScaleDEM')
BOUNDARY_PATH     = os.path.join(data_path, 'derivedFeatures', 'boundary')
AOI_SHP_PATH      = os.path.join(data_path, 'aoi', 'aoi.shp')
BBOX_SHP_PATH     = os.path.join(data_path, 'aoi', 'bbox.shp')

##### Ensure that all the folders exists

In [ ]:
if os.path.isdir(os.path.dirname(FEATURES_PATH)):
    shutil.rmtree(os.path.dirname(FEATURES_PATH))
if os.path.isdir(os.path.dirname(BBOX_SHP_PATH)):
    shutil.rmtree(os.path.dirname(BBOX_SHP_PATH))
nx.ensure_dir(os.path.join(FEATURES_PATH, "dummy"))
nx.ensure_dir(os.path.join(OPTICAL_PATH, "dummy"))
nx.ensure_dir(os.path.join(DEM_PATH, "dummy"))
nx.ensure_dir(os.path.join(BOUNDARY_PATH, "dummy"))
nx.ensure_dir(BBOX_SHP_PATH)

##### DEM Scale Factors

In [ ]:
NO_DATA_VALUE_DEM = -9999
RESAMPLING_FACTOR_DEM = [1, 2, 3, 4]

##### Sentinel-2 Images Parameters

In [ ]:
split_id = s2_identifier.split("_")
IMG_STRING = "_".join([split_id[-2], split_id[2]])

In [ ]:
CREATE_RGB_RESCALED = True
R_MIN_MAX_CH = [ 300 , 1200, 'R' ]
B_MIN_MAX_CH = [ 800 , 1200, 'B' ]
G_MIN_MAX_CH = [ 450 , 1100, 'G' ]
IR_MIN_MAX_CH= [ 400 , 3800,  '' ]

##### Wetness connected

In [ ]:
WETNESS_T = 9
CENTRE_OF_PIXEL_DISTANCE = 5 
N_CONNECTED_PIXEL_MIN = 125
CLOSE_T = 3

##### Region type

In [ ]:
REGION_TYPE = 'Pred'

##### Pre-Processing outputs

In [ ]:
region = model['value']
if region == 'Oregon':
    inputImages = [[ os.path.join(FEATURES_PATH, 'DEM_007_HILLSHADE.tif'),         1/255.0 ],
                   [ os.path.join(FEATURES_PATH, 'DEM_007_HILLSHADE_45.tif'),      1/255.0 ],
                   [ os.path.join(FEATURES_PATH, 'DEM_007_HILLSHADE_180.tif'),     1/255.0 ],
                   [ os.path.join(FEATURES_PATH, 'DEM_007_ROUGHNESS.tif'),         1/150.0 ],

                   [ os.path.join(FEATURES_PATH, 'DEM_014_SLOPE.tif'),             1/90.0 ], 

                   [ os.path.join(FEATURES_PATH, 'DEM_027_WETNESS_connected.tif'), 1/255.0 ],

                   [ os.path.join(OPTICAL_PATH,  'S2_NDVI_007_UINT8.tif'),         1/255.0 ],

                   [ os.path.join(OPTICAL_PATH,  'S2_B03_007_UINT8.tif'),          1/255.0 ],
                   [ os.path.join(OPTICAL_PATH,  'S2_B04_007_UINT8.tif'),          1/255.0 ],
                   [ os.path.join(OPTICAL_PATH,  'S2_B08_007_UINT8.tif'),          1/255.0 ]]
    
elif region == 'Bhutan':
    inputImages = [[ os.path.join(FEATURES_PATH, 'DEM_030_HILLSHADE.tif'),         1/255.0 ],
                   [ os.path.join(FEATURES_PATH, 'DEM_030_HILLSHADE_45.tif'),      1/255.0 ],
                   [ os.path.join(FEATURES_PATH, 'DEM_030_HILLSHADE_180.tif'),     1/255.0 ],
                   [ os.path.join(FEATURES_PATH, 'DEM_030_ROUGHNESS.tif'),         1/150.0 ],

                   [ os.path.join(FEATURES_PATH, 'DEM_030_SLOPE.tif'),             1/60.0 ], 

                   [ os.path.join(FEATURES_PATH, 'DEM_030_WETNESS_connected.tif'), 1/255.0 ],

                   [ os.path.join(OPTICAL_PATH,  'S2_NDVI_010_UINT8.tif'),         1/255.0 ],

                   [ os.path.join(OPTICAL_PATH,  'S2_B02_010_UINT8.tif'),          1/255.0 ],
                   [ os.path.join(OPTICAL_PATH,  'S2_B03_010_UINT8.tif'),          1/255.0 ],
                   [ os.path.join(OPTICAL_PATH,  'S2_B04_010_UINT8.tif'),          1/255.0 ],
                   [ os.path.join(OPTICAL_PATH,  'S2_B08_010_UINT8.tif'),          1/255.0 ]]

DL_INPUT_PATH = [ ]
DL_SCALE_PARAMETER  = [ ]

for i, j in inputImages:
    DL_INPUT_PATH.append(i)
    DL_SCALE_PARAMETER.append(j)

## Multi-resolution DEM

In [ ]:
import numpy as np
from osgeo import gdal   
import cv2
import math

In [ ]:
ciop.log ("INFO", "Conversion of WKT to SHP")
interpolation_method = cv2.INTER_LINEAR
demPath_diff_px = os.path.join(DEM_PATH, 'temp_diff_px.tif')
demPath_xeqy = os.path.join(DEM_PATH, 'temp_xeqy.tif')

nx.wkt2shp(aoi['value'], PROJECT_EPSG, BBOX_SHP_PATH, True)

In [ ]:
ciop.log ("INFO", "Intersecting DEM data with SHP")
gdalwrapCmd = "/opt/anaconda/envs/p36-ethz-03-01-01/bin/gdalwarp -cutline " + BBOX_SHP_PATH + \
    " -crop_to_cutline -t_srs EPSG:" + str(PROJECT_EPSG) + \
    " -r bilinear -dstnodata " + str(NO_DATA_VALUE_DEM) +  \
    " -of GTiff -overwrite " + str(SOURCE_DEM_PATH) + " " + str(demPath_diff_px) 
os.system(gdalwrapCmd)

In [ ]:
region = model['value']
if region == 'Oregon':
    gdaltranslateCmd = "/opt/anaconda/envs/p36-ethz-03-01-01/bin/gdal_translate -tr 27 27 {} {}"
elif region == 'Bhutan':
    gdaltranslateCmd = "/opt/anaconda/envs/p36-ethz-03-01-01/bin/gdal_translate -tr 30 30 {} {}"

os.system(gdaltranslateCmd.format(str(demPath_diff_px), str(demPath_xeqy)))

In [ ]:
def get_image_transform(image_path):
    # Get information from image
    dataset = gdal.Open(image_path, gdal.GA_ReadOnly)
    projection = dataset.GetProjection()

    geotransform = dataset.GetGeoTransform()
    xSize = dataset.RasterXSize
    ySize = dataset.RasterYSize
    xResolution = geotransform[1]
    yResolution = geotransform[5]

    image = dataset.ReadAsArray()
    image = np.float32(image)
    image[image == NO_DATA_VALUE_DEM] = np.nan
    return image, geotransform, projection

In [ ]:
diff_px_image, diff_px_transform, projection = get_image_transform(demPath_diff_px)
eq_px_image, eq_px_transform, projection = get_image_transform(demPath_xeqy)

In [ ]:
def get_transformation_resolution(resolution):
    return str(math.ceil(resolution)).zfill(3)

In [ ]:
ciop.log ("INFO", "Resampling DEM data")
for scale in RESAMPLING_FACTOR_DEM:
    image = eq_px_image if scale == 1 else diff_px_image
    geotransform = eq_px_transform if scale == 1 else diff_px_transform
    #https://medium.com/@wenrudong/what-is-opencvs-inter-area-actually-doing-282a626a09b3
    newData = cv2.resize(image, None, fx = scale, fy = scale, interpolation=interpolation_method) 
    
    newGeotransform = list(geotransform)
    newGeotransform[1] = geotransform[1]/scale
    newGeotransform[5] = geotransform[5]/scale
    newGeotransform = tuple(newGeotransform)
    filename_template = 'DEM_{}.{}'
    newFileName = os.path.join(DEM_PATH, filename_template.format(get_transformation_resolution(newGeotransform[1]), 'tif'))
    nx.writeNumpyArr2Geotiff(newFileName, newData, geoTransform = newGeotransform, projection = projection, GDAL_dtype = gdal.GDT_Int16, noDataValue = -9999)
    print("Written:",  newFileName)
    newFileNameSaga = os.path.join(DEM_PATH, filename_template.format(get_transformation_resolution(newGeotransform[1]), 'sdat'))
    saga_convert = "/opt/anaconda/envs/p36-ethz-03-01-01/bin/gdal_translate -of SAGA {} {}".format(newFileName, newFileNameSaga)
    print("Running: {}".format(saga_convert))
    os.system(saga_convert)

## Features from DEM

In [ ]:
ciop.log ("INFO", "Computing Hillshade, Roughness and Slope using GDAL")
rasterPaths = nx.getResolution(DEM_PATH, return_full_paths = True)
optGDAL = ['hillshade', 'roughness', 'slope']
arguments = ['', '', '']

for rasterPath in rasterPaths:
    if not os.path.isfile(rasterPath):
        print('Unable to open input file ' + rasterPath)
        continue
    basename = '{}_{}{}'.format(os.path.basename(rasterPath)[:-4], '{}', '{}')
    for opt, arg in zip(optGDAL, arguments):
        outputPath = os.path.join(FEATURES_PATH, basename.format(opt.upper(), '.tif'))
        if outputPath in DL_INPUT_PATH:
            commandGDAL = '/opt/anaconda/envs/p36-ethz-03-01-01/bin/gdaldem ' + opt + ' ' + rasterPath + ' '  + outputPath + ' ' + str(arg)
            print('Running: ' + commandGDAL)
            os.system(commandGDAL)
        
        if opt == 'hillshade':
            for az in [45, 180]:
                variant_ext = "_{}.{}".format(str(az), "tif")
                outputPath = os.path.join(FEATURES_PATH, basename.format(opt.upper(), variant_ext))
                if outputPath in DL_INPUT_PATH:
                    commandGDAL = '/opt/anaconda/envs/p36-ethz-03-01-01/bin/gdaldem ' + opt + ' ' + rasterPath + ' -az ' + str(az) + ' ' + outputPath + ' ' + str(arg)
                    print('Running: ' + commandGDAL)
                    os.system(commandGDAL)


In [ ]:
ciop.log ("INFO", "Computing Wetness using SAGA-GIS")

commandCompound = "saga_cmd ta_compound 0"
outputArgsCompound = ['-WETNESS']

for rasterPath in rasterPaths:
    INPUT = nx.joinStrArg( ' -ELEVATION' , rasterPath[:-4] + '.sgrd') 
    outPath = []
    newArgs = []
    for arg in outputArgsCompound:
        partial_filename = 'DEM_{}'.format(str(WORK_RES).zfill(3))
        tmp_path = os.path.join(FEATURES_PATH, os.path.basename(rasterPath)[:-4] + '_' + arg[1:] + '.sdat')

        if partial_filename in os.path.basename(rasterPath)[:-4]:
            newArgs.append(arg)
            outPath.append(tmp_path)
    ARG = ''
    for x, y in zip(newArgs, outPath):
        ARG = nx.joinStrArg(ARG, nx.joinStrArg(x,y))
    if ARG:
        print("Running: " + commandCompound + INPUT + ARG )
        os.system(commandCompound + INPUT + ARG)

## Rasterize AoI

In [ ]:
from osgeo import gdal, ogr, osr
gdal.UseExceptions()

In [ ]:
resolution = nx.getResolution(DEM_PATH)
nx.wkt2shp(aoi['value'], PROJECT_EPSG, AOI_SHP_PATH)
vectorPath = AOI_SHP_PATH

In [ ]:
ciop.log ("INFO", "Rasterizing the Area of Interest")
for res in resolution:
    rasterPath = os.path.join(DEM_PATH, 'DEM_' + str(res).zfill(3) + '.tif')
    outputPath = os.path.join(BOUNDARY_PATH, REGION_TYPE + '_' + str(res).zfill(3) + '.tif')

    try:
        ds = gdal.Open(rasterPath)
    except RuntimeError:
        print('Unable to open input file')
        sys.exit(1)

    geoTransform = ds.GetGeoTransform()
    projection = ds.GetProjection()
    srs = osr.SpatialReference(wkt=projection)
    nscn, npix = ds.RasterYSize, ds.RasterXSize 
    if srs.IsProjected():
        print('PCS: ', srs.GetAttrValue('projcs'))
    else:
        print('GCS: ', srs.GetAttrValue('geogcs'))

    vs = ogr.Open(vectorPath)
    layer = vs.GetLayer()

    ds_new = gdal.GetDriverByName('GTiff').Create(outputPath, npix, nscn, 1, gdal.GDT_Byte)
    ds_new.SetGeoTransform(ds.GetGeoTransform())
    ds_new.SetProjection(ds.GetProjection())

    outBand = ds_new.GetRasterBand(1)
    outBand.Fill(0)
    ds_new.GetRasterBand(1).SetNoDataValue(0)

    # Rasterize the shapefile layer to our new dataset
    status = gdal.RasterizeLayer(ds_new,  # output to our new dataset
                                 [1],  # output to our new dataset's first band
                                 layer,  # rasterize this layer
                                 None, None,  # don't worry about transformations since we're in same projection
                                 [1],  # burn value 1
                                 ['ALL_TOUCHED=TRUE'])  # rasterize all pixels touched by polygons

    # Close dataset
    ds_new.FlushCache()
    ds_new = None
    outBand = None

    ds = None


## Crop Sentinel-2 Raster to DEM extent

In [ ]:
TMP_S2_WORKSPACE = os.path.join(data_path, 'TMP_S2')
if os.path.isdir(TMP_S2_WORKSPACE):
    shutil.rmtree(os.path.dirname(os.path.join(TMP_S2_WORKSPACE, "dummy")))


In [ ]:
ciop.log ("INFO", "Cropping Sentinel-2 Raster to DEM extent")
for res in S2_RESOLUTION:

    #destRaster = os.path.join(DEM_PATH, 'DEM_' + str(res).zfill(3) + '.tif')
    destRaster = os.path.join(BOUNDARY_PATH, REGION_TYPE + '_' + str(res).zfill(3) + '.tif')
    noData = '-srcnodata 0 -dstnodata 0'
    inputOutputList = []
    for band in ['_B02', '_B03', '_B04', '_B08']:
        source_in = os.path.join(SOURCE_S2_IMAGE_FOLDER, IMG_STRING + band + '.jp2')
        opt_feat  = os.path.join(OPTICAL_PATH, 'S2' + band + '_' + str(res).zfill(3) + '.tif')
        inputOutputList.append([source_in, opt_feat])
    
    
    raster_ds = gdal.Open(destRaster, gdal.GA_ReadOnly)
    
    projection = raster_ds.GetProjection()
    geoTransform = raster_ds.GetGeoTransform()
    epsg = int(nx.wkt2EPSG(projection).split(':')[1])
    
    tgt_srs = osr.SpatialReference()
    tgt_srs.ImportFromWkt(projection)
    
    cornerPoints = nx.getCornerCoordinates(raster_ds, False)
    
    npixDEM = raster_ds.RasterXSize
    nscnDEM = raster_ds.RasterYSize
    
    dem = nx.readGDAL2numpy(destRaster)
    
    # create ring
    ring = ogr.Geometry(ogr.wkbLinearRing)
    for point in cornerPoints:
        lat = point[0]
        long = point[1]
        ring.AddPoint(lat,long)
    
    #add first point again to close ring
    ring.AddPoint(cornerPoints[0][0], cornerPoints[0][1])
    
    #add ring to polygon
    extentPoly = ogr.Geometry(ogr.wkbPolygon)
    extentPoly.AddGeometry(ring)
    del ring
    
    ##Define a geographic coordinate system
    #gcs = osr.SpatialReference()
    #gcs.ImportFromEPSG(epsg)
    
    ##Assign Spatial Refrence to polygon
    extentPoly.AssignSpatialReference(tgt_srs)
    
    nx.ensure_dir(os.path.join(TMP_S2_WORKSPACE, "dummy"))
    
    driver = ogr.GetDriverByName('Esri Shapefile')
    ds = driver.CreateDataSource(os.path.join(TMP_S2_WORKSPACE, 'demExtent_EPSG' + str(epsg) + '.shp'))
    layer = ds.CreateLayer('', None, ogr.wkbPolygon)
    
    # Add an attribute
    layer.CreateField(ogr.FieldDefn('id', ogr.OFTInteger))
    defn = layer.GetLayerDefn()
    
    # Create a new feature
    feat = ogr.Feature(defn)
    feat.SetField('id', 123)
    feat.SetGeometry(extentPoly)
    
    layer.CreateFeature(feat)
    
    feat = None
    ds = layer = feat = None
    
    tgt_srs.MorphToESRI()
    
    file = open(os.path.join(TMP_S2_WORKSPACE, 'demExtent_EPSG' + str(epsg) + '.prj'), 'w')
    file.write(tgt_srs.ExportToWkt())
    file.close()
    
    for raster2clip, outputRasterFileName in inputOutputList:
        if outputRasterFileName is None or  outputRasterFileName == '':
            outputRasterFileName = str(raster2clip[:-4]) + '_reProjected.tif'
        
        assert os.path.exists(raster2clip)
        
        cmdStr = '/opt/anaconda/envs/p36-ethz-03-01-01/bin/gdalwarp -overwrite -t_srs EPSG:' + str(epsg) + ' -r cubic ' + noData + ' -multi -q -cutline ' + os.path.join(TMP_S2_WORKSPACE, 'demExtent_EPSG' + str(epsg) + '.shp') + ' -crop_to_cutline -tr ' \
                    + str(geoTransform[1]) + ' ' + str(geoTransform[5]) + ' -of GTiff ' + str(raster2clip) + ' ' + str(outputRasterFileName)
        print('EXECUTE >> ' + str(cmdStr))
        os.system(cmdStr)
        nx.resizeToDEM(outputRasterFileName, (nscnDEM, npixDEM), geoTransform, projection, noData = 0)
        print
    shutil.rmtree(os.path.dirname(os.path.join(TMP_S2_WORKSPACE, "dummy")))

## Sentinel-2 Derive from Optical

In [ ]:
ciop.log ("INFO", "Computing optical features from Sentinel-2")
for res in S2_RESOLUTION:
    BLUE  = os.path.join(OPTICAL_PATH, 'S2_B02_' + str(res).zfill(3) + '.tif')
    GREEN = os.path.join(OPTICAL_PATH, 'S2_B03_' + str(res).zfill(3) + '.tif')
    RED   = os.path.join(OPTICAL_PATH, 'S2_B04_' + str(res).zfill(3) + '.tif')
    IR    = os.path.join(OPTICAL_PATH, 'S2_B08_' + str(res).zfill(3) + '.tif')
        
    def convert2Float(im):
        im = np.float32(im)
        im[im == 0] = np.nan
        return im        
    
    def convert2Uint16(im):
        im[im is np.nan] = 0
        return np.uint16(im)
    
    def writeNumpyArr2Geotiff_RGBA(outputPath, data, geoTransform = None, projection = None, GDAL_dtype = gdal.GDT_Byte, noDataValue = None):
        b, g, r, a = data
        nscn, npix = b.shape
        
        if np.isnan(data).any() and noDataValue is not None:
            data[np.isnan(data)] = noDataValue
            
        ds_new = gdal.GetDriverByName('GTiff').Create(outputPath, npix, nscn, 1, GDAL_dtype)
        
        if geoTransform != None:
            ds_new.SetGeoTransform(geoTransform)
            
        if projection != None:
            ds_new.SetProjection(projection)    
        
        outBand = ds_new.GetRasterBand(1)
        outBand.WriteArray(data)
        
        if noDataValue != None:
            ds_new.GetRasterBand(1).SetNoDataValue(noDataValue)
        
        # Close dataset
        ds_new.FlushCache()
        ds_new = None
        outBand = None
        
        
    ir, geoTransform, projection = nx.readGDAL2numpy(IR, True)
    ir = convert2Float(ir)
    
    b = convert2Float(nx.readGDAL2numpy(BLUE, False))
    g = convert2Float(nx.readGDAL2numpy(GREEN, False))
    r = convert2Float(nx.readGDAL2numpy(RED, False))
    
    ndvi = (ir - r)/(ir + r)
    
    nx.writeNumpyArr2Geotiff(os.path.join(OPTICAL_PATH, 'S2_NDVI_' + str(res).zfill(3) + '.tif'), ndvi, geoTransform = geoTransform, \
                             projection = projection, GDAL_dtype = gdal.GDT_Float32, noDataValue = -9999)
    
        
    ir = convert2Uint16(ir)
    r = convert2Uint16(r)
    b = convert2Uint16(b)
    g = convert2Uint16(g)
    
    nscn, npix = b.shape
    ds_new = gdal.GetDriverByName('GTiff').Create(os.path.join(OPTICAL_PATH, 'S2_RGBA_' + str(res).zfill(3) + '.tif'), npix, nscn, 4, gdal.GDT_UInt16)
    
    if geoTransform != None:
        ds_new.SetGeoTransform(geoTransform)
        
    if projection != None:
        ds_new.SetProjection(projection)    
    
    outBand = ds_new.GetRasterBand(1)
    outBand.WriteArray(ir)
    ds_new.GetRasterBand(1).SetNoDataValue(0)
    
    outBand = ds_new.GetRasterBand(2)
    outBand.WriteArray(r)
    ds_new.GetRasterBand(2).SetNoDataValue(0)
    
    outBand = ds_new.GetRasterBand(3)
    outBand.WriteArray(g)
    ds_new.GetRasterBand(3).SetNoDataValue(0)
    
    outBand = ds_new.GetRasterBand(4)
    outBand.WriteArray(b)
    ds_new.GetRasterBand(4).SetNoDataValue(0)
    
    # Close dataset
    ds_new.FlushCache()
    ds_new = None
    outBand = None

## Rescale Optical Bands

In [ ]:
ciop.log ("INFO", "Rescaling optical features")
for res in S2_RESOLUTION:
    
    data = [    
            [os.path.join(OPTICAL_PATH, 'S2_B02_' + str(res).zfill(3) + '.tif'), B_MIN_MAX_CH[0] ,  B_MIN_MAX_CH[1],  B_MIN_MAX_CH[2]],
            [os.path.join(OPTICAL_PATH, 'S2_B03_' + str(res).zfill(3) + '.tif'), G_MIN_MAX_CH[0] ,  G_MIN_MAX_CH[1],  G_MIN_MAX_CH[2]],
            [os.path.join(OPTICAL_PATH, 'S2_B04_' + str(res).zfill(3) + '.tif'), R_MIN_MAX_CH[0] ,  R_MIN_MAX_CH[1],  R_MIN_MAX_CH[2]],
            [os.path.join(OPTICAL_PATH, 'S2_B08_' + str(res).zfill(3) + '.tif'), IR_MIN_MAX_CH[0] , IR_MIN_MAX_CH[1], IR_MIN_MAX_CH[2]] 
                ]
            
    data_ndvi_path = os.path.join(OPTICAL_PATH, 'S2_NDVI_' + str(res).zfill(3) + '.tif')
    
    
    ####################### ############ #######################
    ####################### Rescale NDVI #######################
    ####################### ############ #######################
    
    image, geoT, proj = nx.readGDAL2numpy(data_ndvi_path, True)
    
    image = (image * 100).astype(np.int16)
    image[image <= 0 ] = 0
    image[image >= 80] = 80
    image = image.astype(np.uint16)
    
    image = nx.map_uint16_to_uint8(image, 0, 80)
    
    nx.writeNumpyArr2Geotiff(data_ndvi_path[:-4] + '_UINT8.tif', image, geoTransform = geoT, \
                             projection = proj, GDAL_dtype = gdal.GDT_Byte, noDataValue = None)
    
    del image, data_ndvi_path, geoT, proj
    
    
    ####################### ############# #######################
    ####################### Rescale bands #######################
    ####################### ############# #######################
    for imPath, loVal, hiVal, ch in data:
        
        image, geoT, proj = nx.readGDAL2numpy(imPath, True)
        image = nx.map_uint16_to_uint8(image, loVal, hiVal)
        
        nx.writeNumpyArr2Geotiff(imPath[:-4] + '_UINT8.tif', image, geoTransform = geoT, \
                             projection = proj, GDAL_dtype = gdal.GDT_Byte, noDataValue = None)
         
            
        if ch == 'R':
            r = image.copy()
            geoT2 = geoT
            proj2 = proj
        if ch == 'G':
            g = image.copy()
        if ch == 'B':
            b = image.copy()
        
        
    createRGB = 0
    
    
    for imPath, loVal, hiVal, ch in data:
        if ch == 'R' or ch == 'B' or ch == 'G':
            createRGB += 1
    
    if CREATE_RGB_RESCALED:        
        if createRGB == 3:
            createRGB = True
            print("RGB image will be created")
        else:
            createRGB = False
            print("RBG channels not correctly defined. RGB image will not be created.")        
    else:
        createRGB = False
    
    
    
    if createRGB:
        nscn, npix = b.shape
        ds_new = gdal.GetDriverByName('GTiff').Create(os.path.join(OPTICAL_PATH, 'S2_RGB_' + str(res).zfill(3) + '_UINT8.tif'), npix, nscn, 3, gdal.GDT_Byte)
        
        if geoT2 != None:
            ds_new.SetGeoTransform(geoT2)
            
        if proj2 != None:
            ds_new.SetProjection(proj2)    
        
        
        outBand = ds_new.GetRasterBand(1)
        outBand.WriteArray(r)
        # ds_new.GetRasterBand(1).SetNoDataValue(0)
        
        outBand = ds_new.GetRasterBand(2)
        outBand.WriteArray(g)
        # ds_new.GetRasterBand(2).SetNoDataValue(0)
        
        outBand = ds_new.GetRasterBand(3)
        outBand.WriteArray(b)
        # ds_new.GetRasterBand(3).SetNoDataValue(0)
        
        # Close dataset
        ds_new.FlushCache()
        ds_new = None
        outBand = None    
    
    del createRGB

## Clean Optical Features

In [ ]:
files = [os.path.join(OPTICAL_PATH, f) for f in os.listdir(OPTICAL_PATH) if (os.path.isfile(os.path.join(OPTICAL_PATH, f)) and os.path.join(OPTICAL_PATH, f) not in DL_INPUT_PATH)]
for file in files:
    os.remove(file)

## Wetness Threshold

In [ ]:
from skimage import morphology

In [ ]:
ciop.log ("INFO", "Computing wetness feature")
filename_template = 'DEM_{}_WETNESS{}'.format(str(WORK_RES).zfill(3), '{}')

file_id = '.sdat'
filename = filename_template.format(file_id)
file_path = os.path.join(FEATURES_PATH, filename)

wetness, geoTransform, projection = nx.readGDAL2numpy(file_path, True)
nscn, npix = wetness.shape
wetness[np.isnan(wetness)] = 0

wetness[wetness < WETNESS_T] = 0
wetness[wetness >= WETNESS_T] = 1
wetness = np.array(wetness, np.uint8)

file_id = '_threshold.tif'
filename = filename_template.format(file_id)
file_path = os.path.join(FEATURES_PATH, filename)
nx.writeNumpyArr2Geotiff(file_path, wetness * 255, geoTransform = geoTransform, projection = projection, GDAL_dtype = gdal.GDT_Byte, noDataValue = 0)

wetness = morphology.remove_small_objects(wetness.astype(np.bool), min_size = N_CONNECTED_PIXEL_MIN, connectivity=CENTRE_OF_PIXEL_DISTANCE).astype(np.uint8)
wetness = nx.closeCV(wetness, CLOSE_T)

file_id = '_connected.tif'
filename = filename_template.format(file_id)
file_path = os.path.join(FEATURES_PATH, filename)
nx.writeNumpyArr2Geotiff(file_path, wetness * 255, geoTransform = geoTransform, projection = projection, GDAL_dtype = gdal.GDT_Byte, noDataValue = 0)

## Classification parameters

In [ ]:
region = model['value']
if region == 'Oregon':
    RESOLUTION = 7
elif region == 'Bhutan':
    RESOLUTION = 10
OUTPUT_FOLDER_NAME  = data_path #os.path.join(data_path, 'outputFolder')
mask_template = "{}_{}.tif"
BOUNDARY_MASK = os.path.join(BOUNDARY_PATH, mask_template.format(REGION_TYPE, str(RESOLUTION).zfill(3)))
LOCAL_MODEL_PATH = "./{}".format(model_name)
APP_MODEL_PATH = "/application/notebook/libexec/{}".format(model_name)
SCALE_FACTOR = 1
TILE_SIZE = 224
THRESHOLD = 50

In [ ]:
import itertools
from tensorflow.keras import models
from tensorflow.keras import backend as K
from tensorflow.python.keras import optimizers
from losses import BCE_F_TI_LOSS, MCC, PRED_AREA, POD, POFD 

In [ ]:
def bbox(img):
    scn = np.any(img, axis=1)
    pix = np.any(img, axis=0)
    scnMin, scnMax = np.where(scn)[0][[0, -1]]
    pixMin, pixMax = np.where(pix)[0][[0, -1]]
    return [scnMin, scnMax, pixMin, pixMax]


def getBoundingBox(resolution, regionType = REGION_TYPE, returnBinaryMask = False):
    rasterPath = os.path.join(BOUNDARY_PATH, REGION_TYPE + '_' + str(resolution).zfill(3) + '.tif')
    maskImage = nx.readGDAL2numpy(rasterPath, False)

    yMin, yMax, xMin, xMax = bbox(maskImage)

    if returnBinaryMask:
        return yMin, yMax, xMin, xMax , maskImage[yMin : yMax, xMin : xMax]
    else:
        return yMin, yMax, xMin, xMax

### TODO: Check if all required pre-processed files are available

In [ ]:
ciop.log ("INFO", "Loading features into memory")
inputChannel = len(DL_INPUT_PATH)

mask, geoTransform, projection = nx.readGDAL2numpy(BOUNDARY_MASK, return_geoInformation = True)
inputImages = np.zeros([mask.shape[0], mask.shape[1], inputChannel], dtype = np.float32)
for i in range(inputChannel):
    inputImages[ : , : , i] = cv2.resize(nx.readGDAL2numpy(DL_INPUT_PATH[i], return_geoInformation = False), (mask.shape[1], mask.shape[0]), interpolation=cv2.INTER_CUBIC)
    ciop.log ("INFO", DL_INPUT_PATH[i] + " loaded in Memory.")
del i

yMin, yMax, xMin, xMax, binaryMask = getBoundingBox(resolution = RESOLUTION, regionType = REGION_TYPE, returnBinaryMask = True)

testImage = inputImages[yMin : yMax, xMin : xMax, :]

del inputImages, mask

In [ ]:
fetchSize = int(TILE_SIZE / 2)

Y = [y for y in range(fetchSize, testImage.shape[0] - fetchSize, fetchSize)]
X = [x for x in range(fetchSize, testImage.shape[1] - fetchSize, fetchSize)]


newGeoTransform = nx.newGeoTransform(geoTransform, {'xMin' : xMin, 'yMin' : yMin})
shrinkGeoTransform = nx.shrinkGeoTransform(newGeoTransform, 1/SCALE_FACTOR)

In [ ]:
ciop.log ("INFO", "Loading model into memory")
if os.path.isfile(LOCAL_MODEL_PATH):
    model = models.load_model(LOCAL_MODEL_PATH, compile=False)
elif os.path.isfile(APP_MODEL_PATH):
    model = models.load_model(APP_MODEL_PATH, compile=False)
else:
    sys.exit()

In [ ]:
make_loss = BCE_F_TI_LOSS()
model_optimizer = optimizers.Adam()


model.compile(optimizer=model_optimizer, loss=make_loss, metrics=[MCC, PRED_AREA, POD, POFD])
      
shrinkLabelImageShape = [np.int(testImage.shape[0]/SCALE_FACTOR), np.int(testImage.shape[1]/SCALE_FACTOR)]

outputTileSize = int(TILE_SIZE / 2 * SCALE_FACTOR)
outputFetchSize = int(outputTileSize/2)

predictMask = np.zeros(shrinkLabelImageShape, np.uint8)
predictStable = np.zeros(shrinkLabelImageShape, np.uint8)

In [ ]:
ciop.log ("INFO", "Computing prediction")
img_shape = (TILE_SIZE, TILE_SIZE, inputChannel)


locXY = itertools.product(X,Y)
for x, y in locXY:
    img = testImage[y - fetchSize : y + fetchSize, x - fetchSize : x + fetchSize, :] * DL_SCALE_PARAMETER
    img = img.reshape(1, img_shape[0], img_shape[1], inputChannel)
    predicted_label = model.predict(img)[0]
    
    predictMask[int(y/SCALE_FACTOR) - outputFetchSize : int(y/SCALE_FACTOR) + outputFetchSize, int(x/SCALE_FACTOR) - outputFetchSize : int(x/SCALE_FACTOR) + outputFetchSize] = predicted_label[ outputTileSize-outputFetchSize : outputTileSize+outputFetchSize , outputTileSize-outputFetchSize: outputTileSize+outputFetchSize, 0] * 100   


In [ ]:
ciop.log ("INFO", "Saving results into file")
filename = "{}_LS_PROBABILITY.tif".format(IMG_STRING)
nx.writeNumpyArr2Geotiff(filename, predictMask, shrinkGeoTransform, projection, GDAL_dtype = gdal.GDT_Byte, noDataValue = 0)

del model
K.clear_session()

del testImage, predictMask, predictStable, binaryMask, img    

In [ ]:
landslide_Predict = nx.readGDAL2numpy(filename, False)
landslide_Predict = np.where(landslide_Predict > THRESHOLD, 1, 0)
filename = "{}_LS_PREDICTION.tif".format(IMG_STRING)
nx.writeNumpyArr2Geotiff(filename, landslide_Predict, shrinkGeoTransform, projection, GDAL_dtype = gdal.GDT_Byte, noDataValue = 0)

## Clean Workspace

In [ ]:
CLEAN_PARENT_FOLDER = [FEATURES_PATH, AOI_SHP_PATH]
for folder in CLEAN_PARENT_FOLDER:
    shutil.rmtree(os.path.dirname(folder))